# Validation of all methods
## combine labelling methods and model update methods which are proved to be effective
## that is to say, validation of model update methods using semi-auto labelled data

# ------------------------------------------------------------------------------------------

## Prepare

In [ ]:
#import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model, load_model

from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Input, Lambda
from tensorflow.keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard

import matplotlib.pyplot as plt
import plotly.express as px
import glob
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import time
from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.spatial import distance

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix, precision_score
import seaborn as sns

import tensorflow.keras.backend as K
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from keras.models import Model
from gradcamutils import GradCam, GradCamPlusPlus, ScoreCam, GuidedBackPropagation, superimpose, read_and_preprocess_img, build_guided_model

from affine_a_method_det import set_deterministic, fit_affine_A_method_deterministic, inverse_transform_ridge, mahalanobis_sq

In [ ]:
#fuction for making label
def create_image_labels(n_light, n_water, n_blackline, n_discoloration, n_dotsonline, n_adhesion, n_scratch):
    """
    function for making label corresponding to each abnormal mode
    
    Args:
        n_light (int): the number of light images
        n_water (int): the number of water images
        n_blackline (int): the number of black line images
        n_discoloration (int): the number of discoloration images
        n_dotsonline (int): the number of dots on line images
        n_adhesion (int): the number of adhesion images
        n_scrtch (int): the number of surface scratches images
    
    Returns:
        pd.DataFrame: 1collumn=image、2collumn=label
    """
    # Name list for each abnormal mode
    light_data = [f"Light-{i}" for i in range(1, n_light + 1)]
    water_data = [f"Water-{i}" for i in range(1, n_water + 1)]
    blackline_data = [f"BlackLine-{i}" for i in range(1, n_blackline + 1)]
    discoloration_data = [f"Discoloration-{i}" for i in range(1, n_discoloration + 1)]
    dotsonline_data = [f"DotsOnLine-{i}" for i in range(1, n_dotsonline + 1)]
    copper_data = [f"Adhesion-{i}" for i in range(1, n_adhesion + 1)]
    spark_data = [f"SurfaceScratch-{i}" for i in range(1, n_scratch + 1)]

    # Label list
    light_labels = [0] * n_light
    water_labels = [1] * n_water
    blackline_labels = [2] * n_blackline
    discoloration_labels = [3] * n_discoloration
    dotsonline_labels = [4] * n_dotsonline
    adhesion_labels = [5] * n_adhesion
    scratch_labels = [6] * n_scratch

    # make dataframe by combining data
    data = list(zip(light_data + water_data + blackline_data + discoloration_data + dotsonline_data + copper_data + spark_data, 
                    light_labels + water_labels + blackline_labels + discoloration_labels + dotsonline_labels + copper_labels + spark_labels))
    df = pd.DataFrame(data, columns=["image", "label"])

    return df

def load_7mode_images(src_dirs):
    """
    src_dirs: list containing full pass of seven folders
              ex: [
                   r"C:/path/mode0/*jpg",
                   r"C:/path/mode1/*jpg",
                   ...
                  ]

    return:
      all_images      :list of all images(cv2) 
      images_by_class : list of each abnormal mode images [list0, list1, ..., list6]
      labels          : list of labels corresponding to each image
      nums            : list of the number of each class images [n0,n1,...,n6]
    """

    assert len(src_dirs) == 7, "Designate 7 folder pass"

    images_by_class = []
    nums = []

    # process
    for i, path in enumerate(src_dirs):
        filepaths = glob.glob(path)
        print(f"Class {i}: {len(filepaths)} files")

        imgs = []
        for fp in filepaths:
            img = cv2.imread(fp)
            if img is not None:
                imgs.append(img)

        images_by_class.append(imgs)
        nums.append(len(imgs))

    # combine
    all_images = []
    for cls_imgs in images_by_class:
        all_images.extend(cls_imgs)

    # make label
    labels = create_image_labels(nums) 

    return all_images, images_by_class, labels, nums


In [3]:
# load base model
base_model = load_model('BaseModel.h5')

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
#get pass

#wire raw images
#TrainingData
source_dirs = [
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
]
#TestData
source_test_dirs = [
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
]

#bright images
#TrainingData
target1_dirs = [
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
]
#TestData
test_target1_dirs = [
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
]

#camera dust images
#TrainingData
target2_dirs = [
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
]
#TestData
test_target2_dirs = [
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
    r"C:\Users\pass",
]


In [ ]:
#get data and label
source_all_imgs, source_imgs_by_class, source_label, source_num = load_7mode_images(source_dirs)
source_test_all_imgs, source_test_imgs_by_class, source_test_label, source_test_num = load_7mode_images(source_test_dirs)
target1_all_imgs, target1_imgs_by_class, target1_label, target1_num = load_7mode_images(target1_dirs)
test_target1_all_imgs, test_target1_imgs_by_class, test_target1_label, test_target1_num = load_7mode_images(test_target1_dirs)
target2_all_imgs, target2_imgs_by_class, target2_label, target2_num = load_7mode_images(target2_dirs)
test_target2_all_imgs, test_target2_imgs_by_class, test_target2_label, test_target2_num = load_7mode_images(test_target2_dirs)

In [ ]:
#normalization of images
sourcefile_list = [file.astype(float)/255 for file in source_all_imgs]
sourcefile_list = [cv2.resize(file, (360, 270)) for file in sourcefile_list]
sourcefile_test_list = [file.astype(float)/255 for file in source_test_all_imgs]
sourcefile_test_list = [cv2.resize(file, (360, 270)) for file in sourcefile_test_list]
targetfile1_list = [file.astype(float)/255 for file in target1_all_imgs]
targetfile1_list = [cv2.resize(file, (360, 270)) for file in targetfile1_list]
test_targetfile1_list = [file.astype(float)/255 for file in test_target1_all_imgs]
test_targetfile1_list = [cv2.resize(file, (360, 270)) for file in test_targetfile1_list]
targetfile2_list = [file.astype(float)/255 for file in target2_all_imgs]
targetfile2_list = [cv2.resize(file, (360, 270)) for file in targetfile2_list]
test_targetfile2_list = [file.astype(float)/255 for file in test_target2_all_imgs]
test_targetfile2_list = [cv2.resize(file, (360, 270)) for file in test_targetfile2_list]

#numpy list
original_source_label = source_label["label"]
original_source_label = np.array(original_source_label)
original_source_test_label = source_test_label["label"]
original_source_test_label = np.array(original_source_test_label)
original_target1_label = target1_label["label"]
original_target1_label = np.array(original_target1_label)
original_test_target1_label = test_target1_label["label"]
original_test_target1_label = np.array(original_test_target1_label)
original_target2_label = target2_label["label"]
original_target2_label = np.array(original_target2_label)
original_test_target2_label = test_target2_label["label"]
original_test_target2_label = np.array(original_test_target2_label)


#dummy parameter 
source_label = to_categorical(source_label["label"])
source_test_label = to_categorical(source_test_label["label"])
target1_label = to_categorical(target1_label["label"])
test_target1_label = to_categorical(test_target1_label["label"])
target2_label = to_categorical(target2_label["label"])
test_target2_label = to_categorical(test_target2_label["label"])

#change the data to numpy list
#save original data
raw_sourcefile_list = sourcefile_list
raw_sourcefile_test_list = sourcefile_test_list
raw_targetfile1_list = targetfile1_list
raw_test_targetfile1_list = test_targetfile1_list
raw_targetfile2_list = targetfile2_list
raw_test_targetfile2_list = test_targetfile2_list

#numpy list
sourcefile_list = np.array(sourcefile_list)
sourcefile_test_list = np.array(sourcefile_test_list)
targetfile1_list = np.array(targetfile1_list)
test_targetfile1_list = np.array(test_targetfile1_list)
targetfile2_list = np.array(targetfile2_list)
test_targetfile2_list = np.array(test_targetfile2_list)

In [7]:
#import label output from semi-auto labelling mrethods
#bright images
# 1. import label csv
labels_path1 = r"C:\Users\pass\example.csv"
df_labels1 = pd.read_csv(labels_path1, encoding="utf-8-sig")

# 2. get label list
pred_labels1 = df_labels1["pred_label"].to_numpy(dtype=np.int64)
print(pred_labels1.shape, pred_labels1[:10])
pred_labels1 = to_categorical(pred_labels1)

#camera dust images
# 1. import label csv
labels_path2 = r"C:\Users\pass\example.csv"
df_labels2 = pd.read_csv(labels_path1, encoding="utf-8-sig")

# 2. get label list
pred_labels2 = df_labels2["pred_label"].to_numpy(dtype=np.int64)
print(pred_labels2.shape, pred_labels2[:10])
pred_labels2 = to_categorical(pred_labels2)

(1680,) [0 0 0 3 0 0 0 0 0 0]
(1680,) [0 0 0 0 0 0 0 0 0 0]


# ------------------------------------------------------------------------------------------

## Proposed model update method

In [8]:
#Choose High Score Block and last block(Block13)
layer_names = ['conv_pw_6_relu','conv_pw_13_relu']
selected_layers = [base_model.get_layer(name).output for name in layer_names]
#mking new model
new_model = keras.Model(inputs = base_model.input, outputs = selected_layers)
x = [layers.GlobalAveragePooling2D()(layer) for layer in selected_layers]
x = layers.concatenate(x, axis = -1)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.6)(x)
output = layers.Dense(7, activation="softmax")(x) 
#making final model
final_model = Model(inputs = new_model.input, outputs = output)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 270, 360, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 135, 180, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 135, 180, 32  128         ['conv1[0][0]']                  
                                )                                                           

In [ ]:
#compile
final_model.compile(optimizer=SGD(learning_rate=1e-3, momentum=0.9), loss = "categorical_crossentropy", metrics = ["accuracy"])

final_model.trainable = True
    
for layer in final_model.layers:
    if not isinstance(layer, Dense):  
        layer.trainable = False
        
for i in final_model.layers:
    print(i.name, i.trainable)

print(final_model.summary())

In [ ]:
#EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5)

X_train, X_val, y_train, y_val = train_test_split(
    targetfile1_list, pred_labels1,
    test_size=0.1,         
    stratify=pred_labels1,            
)

#retraining
history = final_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    verbose=1,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

In [ ]:
#training curve
metrics = ['loss', 'accuracy']
plt.figure(figsize=(10,5))

for i in range(len(metrics)):
    metric=metrics[i]
    plt.subplot(1,2,i+1)
    plt.title(metric)
    
    plt_train = history.history[metric]
    plt_test = history.history['val_'+metric]
    
    plt.plot(plt_train, label='training')
    plt.plot(plt_test, label='test')
    plt.legend()
    
plt.show

In [ ]:
#evaluating updated model
predictions = final_model.predict(test_targetfile1_list, batch_size=8)
results = predictions.argmax(axis = 1)
cm = confusion_matrix(original_test_target1_label, results)
sns.heatmap(cm, annot=True, cmap='Blues')

# ------------------------------------------------------------------------------------------

## Relearning

In [ ]:
# --- prepare data ---

X = targetfile1_list
y = pred_labels1

# --- train/val split ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.1,
    stratify=y,
    random_state=42
)

for layer in base_model.layers:
    layer.trainable = True

# --- compile ---
base_model.compile(
    optimizer=SGD(learning_rate=1e-4, momentum=0.9),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

base_model.summary()

for layer in base_model.layers:
    print(layer.name, layer.trainable)


In [ ]:
# --- EarlyStopping ---
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5)

# --- retraining ---
history = base_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=16,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
#training curve

metrics = ['loss', 'accuracy']
plt.figure(figsize=(10,5))

for i in range(len(metrics)):
    metric=metrics[i]
    plt.subplot(1,2,i+1)
    plt.title(metric)
    
    plt_train = history.history[metric]
    plt_test = history.history['val_'+metric]
    
    plt.plot(plt_train, label='training')
    plt.plot(plt_test, label='test')
    plt.legend()
    
plt.show

In [ ]:
#evaluating updated model
predictions = base_model.predict(test_targetfile1_list, batch_size=8)
results = predictions.argmax(axis = 1)
cm = confusion_matrix(original_test_target1_label, results)
sns.heatmap(cm, annot=True, cmap='Blues')